In [1]:
import requests as r
import pandas as pd
import numpy as np
import os

import sys
sys.path.append("../../helper_functions")
import duneapi_utils as du
import pandas_utils as p
import google_bq_utils as bqu
sys.path.pop()

import time

In [2]:
table_name = 'op_collective_revenue_transactions'

trailing_days = 365

In [3]:
# Run Dune
print('     dune runs')
days_param = du.generate_query_parameter(input=trailing_days,field_name='trailing_days',dtype='number')
dune_df = du.get_dune_data(query_id = 3046107, #https://dune.com/queries/3046107
    name = table_name,
    path = "outputs",
    performance="medium",
    params = [days_param],
    num_hours_to_rerun=4
)
dune_df['source'] = 'dune'
dune_df['tx_block_time'] = pd.to_datetime(dune_df['tx_block_time']).dt.tz_localize(None)

     dune runs
Results available at https://dune.com/queries/3046107?trailing_days=365
getting latest result


2025-11-03 14:30:01,661 INFO dune_client.api.base executing 3046107 on medium cluster
2025-11-03 14:30:01,969 INFO dune_client.api.base waiting for query execution 01K95K59GW0GEYDSWTYJXTZXEY to complete: ExecutionState.PENDING (queue position: None)
2025-11-03 14:30:03,345 INFO dune_client.api.base waiting for query execution 01K95K59GW0GEYDSWTYJXTZXEY to complete: ExecutionState.PENDING (queue position: None)
2025-11-03 14:30:04,492 INFO dune_client.api.base waiting for query execution 01K95K59GW0GEYDSWTYJXTZXEY to complete: ExecutionState.EXECUTING
2025-11-03 14:30:05,634 INFO dune_client.api.base waiting for query execution 01K95K59GW0GEYDSWTYJXTZXEY to complete: ExecutionState.EXECUTING
2025-11-03 14:30:06,797 INFO dune_client.api.base waiting for query execution 01K95K59GW0GEYDSWTYJXTZXEY to complete: ExecutionState.EXECUTING
2025-11-03 14:30:07,949 INFO dune_client.api.base waiting for query execution 01K95K59GW0GEYDSWTYJXTZXEY to complete: ExecutionState.EXECUTING
2025-11-03 14:

In [4]:
dune_df['value'] = dune_df['value'].astype(str)
dune_df['chain_id'] = dune_df['chain_id'].astype(str)

### Inspect duplicates for BigQuery merge keys

Use the cell below to find rows and key groups that duplicate the unique key `['tx_block_number', 'tx_hash', 'trace_address']`. It prints a summary, previews the largest duplicate groups, and writes CSVs to `outputs/` for deeper inspection.


In [ ]:
# Find duplicate keys and save details for review
unique_keys = ['tx_block_number', 'tx_hash', 'trace_address']

# Rows participating in duplicate keys
duplicate_mask = dune_df.duplicated(subset=unique_keys, keep=False)
duplicate_rows = dune_df.loc[duplicate_mask].copy()

# Group sizes for duplicate key groups
group_sizes = (
    duplicate_rows
    .groupby(unique_keys, dropna=False)
    .size()
    .reset_index(name='duplicate_count')
    .sort_values('duplicate_count', ascending=False)
)

print(f"Total rows: {len(dune_df):,}")
print(f"Rows with duplicated keys: {len(duplicate_rows):,}")
print(f"Number of duplicate key groups: {group_sizes.shape[0]:,}")

# Preview the largest duplicate groups
try:
    display(group_sizes.head(20))
except NameError:
    print(group_sizes.head(20))

# Preview rows from the largest duplicate group (if any)
if not group_sizes.empty:
    first_group = group_sizes.iloc[0]
    example_filter = (
        (dune_df['tx_block_number'] == first_group['tx_block_number']) &
        (dune_df['tx_hash'] == first_group['tx_hash']) &
        (dune_df['trace_address'] == first_group['trace_address'])
    )
    example_rows = dune_df.loc[example_filter]
    print("\nExample rows from the largest duplicate group:")
    try:
        display(example_rows)
    except NameError:
        print(example_rows)

# Export CSVs for deeper inspection
outputs_dir = 'outputs'
os.makedirs(outputs_dir, exist_ok=True)
group_sizes.to_csv(os.path.join(outputs_dir, 'op_collective_revenue_duplicate_groups.csv'), index=False)
duplicate_rows.to_csv(os.path.join(outputs_dir, 'op_collective_revenue_duplicate_rows.csv'), index=False)
print(f"Saved: {os.path.join(outputs_dir, 'op_collective_revenue_duplicate_groups.csv')}")
print(f"Saved: {os.path.join(outputs_dir, 'op_collective_revenue_duplicate_rows.csv')}")


In [5]:
#BQ Upload
bqu.append_and_upsert_df_to_bq_table(dune_df, table_name, unique_keys = ['tx_block_number','tx_hash','trace_address'])

2025-11-03 14:32:51,604 INFO root Attempt 1: Using service account credentials
2025-11-03 14:32:51,604 WARNING root Service account init failed on attempt 1: [Errno 2] No such file or directory: '/Users/michaelsilberling/Documents/bq/oplabs-tools-data-0842f34ceb69.json'
2025-11-03 14:32:51,605 INFO root Attempt 1: Using ADC/OIDC login
2025-11-03 14:32:53,351 INFO root Attempt 1: Using service account credentials
2025-11-03 14:32:53,352 WARNING root Service account init failed on attempt 1: [Errno 2] No such file or directory: '/Users/michaelsilberling/Documents/bq/oplabs-tools-data-0842f34ceb69.json'
2025-11-03 14:32:53,352 INFO root Attempt 1: Using ADC/OIDC login
2025-11-03 14:32:54,421 INFO root Attempt 1: Using service account credentials
2025-11-03 14:32:54,422 WARNING root Service account init failed on attempt 1: [Errno 2] No such file or directory: '/Users/michaelsilberling/Documents/bq/oplabs-tools-data-0842f34ceb69.json'
2025-11-03 14:32:54,422 INFO root Attempt 1: Using ADC/

Table 'op_collective_revenue_transactions_staging' deleted successfully from dataset 'api_table_uploads'.
Start Writing api_table_uploads.op_collective_revenue_transactions_staging


/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/google/cloud/bigquery/_pandas_helpers.py:484: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


Chunk 1/1 loaded successfully to api_table_uploads.op_collective_revenue_transactions_staging
All data loaded successfully to api_table_uploads.op_collective_revenue_transactions_staging


2025-11-03 14:33:00,639 ERROR google_bq_utils Error during append_and_upsert_df_to_bq_table on attempt 1: 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/oplabs-tools-data/queries/7651df20-4f54-45c8-8095-3394a7828d8d?maxResults=0&location=US&prettyPrint=false: UPDATE/MERGE must match at most one source row for each target row

Location: US
Job ID: 7651df20-4f54-45c8-8095-3394a7828d8d



...

/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/google/cloud/bigquery/_pandas_helpers.py:484: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


Chunk 1/1 loaded successfully to api_table_uploads.op_collective_revenue_transactions_staging
All data loaded successfully to api_table_uploads.op_collective_revenue_transactions_staging


2025-11-03 14:33:18,615 ERROR google_bq_utils Error during append_and_upsert_df_to_bq_table on attempt 3: 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/oplabs-tools-data/queries/a6fbcc7c-fdc9-4ce1-8320-9154bea893ad?maxResults=0&location=US&prettyPrint=false: UPDATE/MERGE must match at most one source row for each target row

Location: US
Job ID: a6fbcc7c-fdc9-4ce1-8320-9154bea893ad



Error on attempt 3: 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/oplabs-tools-data/queries/a6fbcc7c-fdc9-4ce1-8320-9154bea893ad?maxResults=0&location=US&prettyPrint=false: UPDATE/MERGE must match at most one source row for each target row

Location: US
Job ID: a6fbcc7c-fdc9-4ce1-8320-9154bea893ad

All attempts failed


BadRequest: 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/oplabs-tools-data/queries/a6fbcc7c-fdc9-4ce1-8320-9154bea893ad?maxResults=0&location=US&prettyPrint=false: UPDATE/MERGE must match at most one source row for each target row

Location: US
Job ID: a6fbcc7c-fdc9-4ce1-8320-9154bea893ad
